In [ ]:
# Develop a stock screener
# Gap Up strategy screener

In [3]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv
# pip3 install lxml

# load secrets
load_dotenv()

True

In [4]:
# scrape a list of the S&P500 companies from wikipedia table
def get_sp500_tickers_from_wikipedia():
    # wikipedia page that lists S&P 500 companies
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    
    # read the html
    tables = pd.read_html(url)
    
    # easy, scrape the first table 0
    sp500_table = tables[0]
    
    # extract the ticker symbols to a list to use later
    tickers = sp500_table['Symbol'].tolist()
    
    return tickers

sp500_tickers = get_sp500_tickers_from_wikipedia()

# look at the ticker symbols
print(sp500_tickers)

# print the total -> note there are more than 500 (503)
print(len(sp500_tickers))

ImportError: Missing optional dependency 'lxml'.  Use pip or conda to install lxml.

In [9]:
def get_stock_data(symbol):
    iex_key = os.getenv("IEX_API_KEY") 
    
    # define and append endpoints below
    quote_url = f'https://cloud.iexapis.com/v1/stock/{symbol}/quote?token={iex_key}'
    company_url = f'https://cloud.iexapis.com/v1/stock/{symbol}/company?token={iex_key}'
    float_url = f'https://cloud.iexapis.com/v1/data-points/{symbol}/FLOAT?token={iex_key}'
    
    # execute api calls with error handling
    try:
        quote_data = requests.get(quote_url).json()
        company_data = requests.get(company_url).json()
        float_data = requests.get(float_url).json()
    except:
        return None  # Return None if there's an error fetching the data

    # data mapping
    stock_data = {
        'ticker': symbol,
        'stock_price': quote_data.get('latestPrice', None),
        'change_close': quote_data.get('change', None),  # KEY-Price Gap $
        'change_close_pct': quote_data.get('changePercent', None),# KEY-Price Gap %
        'volume_today': quote_data.get('latestVolume', None),
        'volume_avg_5d': None, 
        'market_cap': quote_data.get('marketCap', None),
        'float': float_data,  
        'float_pct': None, 
        'volatility': None,
        'liquidity': None,
        'news': None,
        'sector': company_data.get('sector', None),
        'industry': company_data.get('industry', None),
        'company_name': company_data.get('companyName', None),
        'latestUpdate': quote_data.get('latestUpdate', None)
    }

    return stock_data

# Fetch data for all S&P 500 tickers and store in a list
data_list = []
for ticker in sp500_tickers[:5]:  # TESTING -> limit to first 5
    stock_data = get_stock_data(ticker)
    if stock_data:
        data_list.append(stock_data)

# convert the list of dictionaries -> df
df_stocks = pd.DataFrame(data_list)

df_stocks


,ticker,stock_price,change_close,change_close_pct,volume_today,volume_avg_5d,market_cap,float,float_pct,volatility,liquidity,news,sector,industry,company_name,latestUpdate
0,MMM,100.01,-0.29,-0.00289,2632142,None,55204762924,575964843,None,None,None,None,Manufacturing,Surgical and Medical Instrument Manufacturing,3M Co.,1692388988288
1,AOS,68.92,-0.81,-0.01162,716285,None,10529289321,132401108,None,None,None,None,Manufacturing,Heating Equipment (except Warm Air Furnaces) M...,A.O. Smith Corp.,1692388801968
2,ABT,103.71,-1.22,-0.01163,4809701,None,179973976106,1757325070,None,None,None,None,Manufacturing,Pharmaceutical Preparation Manufacturing,Abbott Laboratories,1692389094506
3,ABBV,150.14,0.12,0.00080,4050854,None,265004108535,1763291410,None,None,None,None,Manufacturing,Pharmaceutical Preparation Manufacturing,Abbvie Inc,1692388993396
4,ACN,302.94,-2.70,-0.00883,1997109,None,201246596092,631712819,None,None,None,None,Information,"Data Processing, Hosting, and Related Services",Accenture plc,1692388801990


In [ ]:
# START 
# you need to re-evaluate the specific vars you need here. 